In [ ]:
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from statistics import mean

In [ ]:

df = pd.read_csv('/content/drive/My Drive/Pm25 Project/combined_2002-2017.csv')
models = {}

future_predictions = {}
mse={}
# Iterate over each location
for (lat, lon), group in df.groupby(['latitude', 'longitude']):
    X = group[['t2m', 'tcc', 'tp', 'wind_speed']]  # Features
    y = group['GWRPM25']  # Target
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Standardize the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train the Random Forest model
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train_scaled, y_train)

    # Predictions on the test set
    y_pred = rf_model.predict(X_test_scaled)

    # Evaluation
    mse[(lat,lon)] = mean_squared_error(y_test, y_pred)

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X, y)

    # Store the trained model
    models[(lat, lon)] = model

#mean square error
mean_sq_err = mean(mse)

In [ ]:
future_df = pd.read_csv('/content/drive/My Drive/Pm25 Project/feature_df.csv')

for (lat, lon), model in models.items():

    X = group[['t2m', 'tcc', 'tp', 'wind_speed']]
    future_weather = future_df(X,index=[(lat,lon)])

    # Predict future PM2.5 values using the model
    future_pm25 = model.predict(future_weather)

    # Store future predictions
    future_predictions[(lat, lon)] = future_pm25


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


TypeError: float() argument must be a string or a real number, not 'ellipsis'

In [ ]:
future_predictions.to_csv('/content/drive/My Drive/Pm25 Project/predicted_2018-2022.csv')